In [5]:
#The following Notebook represents preliminary scraping methods on a single page from steam game store. 
#specificallly for horizon zero dawn (as it was a fantastic game)
#This preliminary scraping is meant to gauge the amount of possible features available, pre API referencing
#I did not focus on cleaning or stripping strings during initial scraping, just wanted to get values.



#from bs4 import BeautifulSoup
#import requests


'''
update - 
realized that the total number of reviews found at the very bottom of the page can only be seen after scrolling.
changing loading of the page from bs to selenium, adding a scroll, then loading the html so see if that helps.
this is being done after all other numbers (except curators which may require a user agent) have been found.
'''
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://store.steampowered.com/app/1151640/Horizon_Zero_Dawn_Complete_Edition/'

user_agent = {'User-agent': 'Mozilla/5.0'}
driver = webdriver.Chrome(chromedriver)
driver.get(url)


In [15]:
#this was before using selenium
url = 'https://store.steampowered.com/app/1151640/Horizon_Zero_Dawn_Complete_Edition/'
response = requests.get(url)

In [6]:
#adding scroll
for i in range(2):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    
    #Wait for page to load
    time.sleep(1)


In [16]:
response.status_code

200

In [147]:
#response.text[:1000]

'<!DOCTYPE html>\r\n<html class=" responsive" lang="en">\r\n<head>\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n\t\t\t<meta name="viewport" content="width=device-width,initial-scale=1">\r\n\t\t<meta name="theme-color" content="#171a21">\r\n\t\t<title>Save 40% on Horizon Zero Dawn™ Complete Edition on Steam</title>\r\n\t<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">\r\n\r\n\t\r\n\t\r\n\t<link href="https://store.akamai.steamstatic.com/public/shared/css/motiva_sans.css?v=Rc2hpzg2Ex3T&amp;l=english" rel="stylesheet" type="text/css" >\n<link href="https://store.akamai.steamstatic.com/public/shared/css/shared_global.css?v=14RNXYQwvlll&amp;l=english" rel="stylesheet" type="text/css" >\n<link href="https://store.akamai.steamstatic.com/public/shared/css/buttons.css?v=6PFqex5UPprb&amp;l=english" rel="stylesheet" type="text/css" >\n<link href="https://store.akamai.steamstatic.com/public/css/v6/store.css?v=3_8obo1e4tX9&amp;l=english" rel="stylesh

In [17]:
#page = response.text #pre selenium

In [22]:
#soup = BeautifulSoup(page, 'html5lib')
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [8]:
print(soup.find("class='apphub_AppName'"))

None


# NAME

In [108]:
###FOUND NAME OF GAME ON TOP OF PAGE

#game_name = soup.find('span',itemprop="name").text
#game_name

#soup.find('div', class_='details_block').find('b', text='Title:').findNext().findNext()

soup.find('div', class_='apphub_AppName').text

'Horizon Zero Dawn™ Complete Edition'

# RELEASE DATE

In [10]:
#FOUND RELEASE DATE 

release_date = soup.find('div', class_='date').text

# CURATORS/BLOGGERS

In [212]:
#!     !     !    !    !    !    !    ! 
#!     !     !    !
#attempting to pull the number of 'curators' - will circle back
#soup.find('div', class_="no_curators_followed")

#soup.find('div', class_='steam_curators_block block responsive_apppage_reviewblock') #another no

#soup.find('h2', text='What Curators Say') #why are these all blank!!

#soup.find('div', class_ = 'block_header') #ok this is generic,maybe we can search within
#soup.find_all('div', class_ = 'block_header')

#soup.find('div', class_="steam_curators_block block responsive_apppage_reviewblock")

#soup.find('div', id="responsive_apppage_reviewblock_ctn").findNext

##soup.find('div', )

#found out must be logged in for this information to populate (thanks ian) so will circle back after using
#selenium

[<h2>About This Game</h2>,
 <h2>System Requirements</h2>,
 <h2 class="bb_tag">PRIVACY POLICY</h2>,
 <h2 class="bb_tag">USER AGREEMENT</h2>,
 <h2>More like this</h2>,
 <h2 class="user_reviews_header no_bottom_margin">Customer reviews</h2>,
 <h2>Popular user-defined tags for this product:<span class="app_tag_modal_tooltip" data-tooltip-text="These are tags applied to the product by the most users.  You can click a tag to find other products with that tag applied.  Or, you can hit the plus symbol for any existing tags to increase that tag's popularity on this product.">(?)</span></h2>,
 <h2>Sign In</h2>]

# NUM REVIEWS

In [48]:
# Number of reviews
#num_reviews = soup.find(class_='user_reviews_summary_bar').find_all('span')[-1].text
#num_reviews
#soup.find('div', id='Reviews_summary', class_='user_reviews_container').find('div', class_='view_all_reviews_btn leftcol')

# found total number of reviews only after scrolling with selenium
#rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
#split_rev = rev_string.split()



#for things in rev_string.split(): #unsure why this was not working exactly.
#    num_reviews=[]
#    if things.isdigit:
#        num_reviews.append(things)  
#num_reviews


#num_reviews = [int(word) for word in rev_string.split() if word.isdigit()]  #why god.
#num_reviews


rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
rev_string.split()[2]

'45,326'

# REVIEW RATING

In [13]:
# rating. their rating system must be rese
rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
rating

'Mostly Positive'

# PUBLISHER

In [109]:
# obtaining publisher name and url to publisher steam page (for future script of number publisher games)

#publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
#publisher_url = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].get('href')

#soup.find('div', class_='details_block').find_all('a')[-1].text # this gave an error while running first 50 pages
                                                                # editing to find a more generalized way
    
publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
publisher

'PlayStation Mobile, Inc.'

# PRICE

In [49]:
#while this does obtain the price from the code, it relies heavily on index 4. uknonwn if will work
#for multiple pages, as there are multiple prices. hopefully the 4th one is always the most general price
#will have to fix

#can also possibly be used for sale purposes in the future/ 

final_price = soup.find('div',class_='game_purchase_action_bg').find_all('div')[4].text

'$29.99'

In [182]:
#comparing price xpaths~
#xpaths do not seem to be working, or i cannot figuure out proper syntax (have not looked up)

#HZD_price_xpath = (soup.find('//*[@id="game_area_purchase_section_add_to_cart_392522"]/div[2]/div/div[1]/div[2]/div[2]'))
#soup.find('id', text='game_area_purchase_section_add_to_cart_392522')

#soup.find('div', id='game_area_purchase')
#soup.find('div', id='leftcol game_description_column')
#soup.find('div', id='game_area_purchase')


#OK i have found this nice string that contains the discount, the original, and the final price
#this can be stripped for those numbers, but I will have to see if applicable to all game pages, 
#otherwise maybe a game price list can be utilized from a separate source

#price_string = soup.find('div', class_='game_purchase_action').text
#stripped_price = price_string.strip('\n')[:16]
#split_price = stripped_price.split('$')
#discount = split_price[0]
#original_price = split_price[1]
#sale_price = split_price[2]
#print(price_string)
#print(split_price)
#print(discount)
#print(original_price)
#print(sale_price)

['-40%', '49.99', '29.99']
-40%
49.99
29.99


In [ ]:
#after fumbling with the price on this page, I have relized the discount, original price, and sale price
#are all obtainable from the page with the list of games, all formatted the same, leaving code for sake
#of seeing workflow

# GENRE

In [59]:
#SO genres are listed in groups, unsure how to process that when coming to regression.
# possible anlysis - use just the firt genre, as it could be considered most important.

#genres are listed alphabetically! this code will only find first genre listed.

genre_first = soup.find('b',text='Genre:').findNext().text
genre_first

'Action'

In [81]:
#does find all genres, but do not know how to seperate for developer
soup.find('div', class_='details_block')

<div class="details_block">
			<b>Title:</b> Horizon Zero Dawn™ Complete Edition<br/>
	
				<b>Genre:</b> <a href="https://store.steampowered.com/genre/Action/?snr=1_5_9__408">Action</a>, <a href="https://store.steampowered.com/genre/Adventure/?snr=1_5_9__408">Adventure</a>, <a href="https://store.steampowered.com/genre/RPG/?snr=1_5_9__408">RPG</a><br/>
			
			<div class="dev_row">
			<b>Developer:</b>

				<a href="https://store.steampowered.com/search/?developer=Guerrilla&amp;snr=1_5_9__408">Guerrilla</a>
		</div>
	
			<div class="dev_row">
			<b>Publisher:</b>

				<a href="https://store.steampowered.com/search/?publisher=PlayStation%20Mobile%2C%20Inc.&amp;snr=1_5_9__422">PlayStation Mobile, Inc.</a>			
		</div>
	
		
			<b>Release Date:</b> Aug 7, 2020<br/>
	
	
	</div>

In [193]:
#we can try calling the whole list except the last 2! assuming there is only one link for developer
#and one link for publisher and they never list collaborations, this will work. future analysis will need 
#to be performed. idealy we could slice unitl new nested class
genres_list = soup.find('div', class_='details_block').find_all('a')[0:-2]
genres_all = []
for links in genres_list:
    genres_all.append(links.text)
# do not know how to pull out this text
print(genres_all)
num_genres = len(genres_all)
print(num_genres)

['Action', 'Adventure', 'RPG']
3


In [240]:
maybe = [link.text for link in soup.find('div', class_='details_block').find_all('a')[0:-2]]
maybe


['Action', 'Adventure', 'RPG']

In [ ]:
##enough for one night. tomorrow, see if all data can be repulled from same tables for uniformity
## compare xpaths for prices on different pages to see if they are usable instead! could be easier.
#still missing languages,tags, number of tags, and bloggers 

# TAGS

In [67]:
#code finds all tags, but again is a find from a find which returns a list, cannot call.txt
tags = soup.find('div', class_='glance_tags').find_all('a')
len(tags)
tags_list = []
#i=0
#for item in genres_all:
#    genre_list.append(genres_all[i].text())
#genres_all[0].text # cannot call text on the list. help.

for items in tags:
    tags_list.append(items.text.strip())
tags_list

['Adventure',
 'Open World',
 'Action',
 'Female Protagonist',
 'RPG',
 'Singleplayer',
 'Story Rich',
 'Post-apocalyptic',
 'Robots',
 'Sci-fi',
 'Third Person',
 'Exploration',
 'Beautiful',
 'Atmospheric',
 'Great Soundtrack',
 'Action RPG',
 'Archery',
 'Hunting',
 'Dinosaurs',
 'Stealth']

# LANGUAGE

In [94]:
#attempting to get language. bottom of the table has the total number of languages. trying to find it

#soup.find('table',class_='game_language_options').findNext()

<tr>
<th style="width: 94px;"></th>
<th class="checkcol">Interface</th>
<th class="checkcol">Full Audio</th>
<th class="checkcol">Subtitles</th>
</tr>

In [57]:
#hoorah! found the string, the number can be extracted tomrrow
languagez = soup.find('a', class_='all_languages').text.split()[2]
languagez

'20'

In [ ]:
##ok really stopping for the night. morning office hours - how to pull the text out of a list, 
# like after finding then finding again, for both genre and tags.

#inquire about the order of using multiple pages, like i have to go from games list to game page to developer
# page, should that all happen at once, or should I run a separate devloper loop?

#ask about language number being a list, is there a better way to extract the information?

In [51]:
language = (word for word in languagez if word.isdigit())

In [52]:
language

<generator object <genexpr> at 0x7fd2f9072a50>

In [121]:
languages_split = languagez.split()
languages_split
language_number = []
for word in languages_split:
    if word.isdigit():
        language_number.append(word)
        x=word
language_number

['20']

In [122]:
x

'20'